In [1]:
import sys, os
from pathlib import Path
import PyPDF2 
from PyPDF2 import PdfFileWriter,PdfFileReader,PdfFileMerger

In [42]:
root = Path('raw/pdfs')

fname = 'msft_highlights.pdf'
# fname = 'sap_highlights.pdf'

path = root / fname

In [43]:
imdir = (root/'imgs')
imdir.mkdir(exist_ok=True)

In [44]:
from pdfstructure.hierarchy.parser import HierarchyParser
from pdfstructure.source import FileSource

parser = HierarchyParser() 

# specify source (that implements source.read())
source = FileSource(path.open('rb')) 

# analyse document and parse as nested data structure
document = parser.parse_pdf(source)

In [45]:
from pdfstructure.printer import PrettyStringPrinter

stringExporter = PrettyStringPrinter()
prettyString = stringExporter.print(document)
print(prettyString)



[Company Research Highlights ®]
	Report created on August 25, 2021

	[MICROSOFT ORD SHS  NASDAQ:MSFT]

		[One Microsoft Way
		Redmond, WA 98052-6399
		USA
		Telephone: 425 882 -8080
		www.microsoft.com]

			[Price on 8/24/2021:]

				[Market Capitalization:]
					$302.62
					$2275.4 B
					Information Technology Sector
					Software Industry
					Business Description  Data provided by S&P Compustat
					Microsoft Corporation develops, licenses, and supports software, services, devices, and
					solutions worldwide.
					3-Year Price Performance  Data provided by DataScope from Refinitiv
			Key Statistics
			Employee Count
			Institutional Ownership
			Total Revenue (TTM)

			[Revenue Growth]
				(TTM vs. Prior TTM)
			Enterprise Value
			Ex. Dividend Date
			Dividend
			P/E (TTM)
			Earning Yield (TTM)
			EPS (Adjusted TTM)
			Dividend Yield (Annualized)

			[Consensus EPS Estimate]
				(Q1 2022)

			[EPS Growth]
				(TTM vs. Prior TTM)
				181,000
				71.7%
				$46,152.00
				6/30/

In [46]:
D = document

In [47]:
D.title

In [48]:
cs = D.elements[0].children#[0]
[c.heading_text for c in cs]

['Report created on August 25, 2021',
 'MICROSOFT ORD SHS  NASDAQ:MSFT',
 'Company Research Highlights®',
 'MSFT',
 'Company Research Highlights®',
 'MSFT',
 'Company Research Highlights®',
 'MSFT']

In [144]:
addr = D.elements[0].children[1].children[0]
addr.heading_text.split('\n')

['One Microsoft Way',
 'Redmond, WA 98052-6399',
 'USA',
 'Telephone: 425 882 -8080',
 'www.microsoft.com']

In [145]:
sector, industry = addr.children[0].children[0].children[-4].heading_text.split('\n')
sector = ' '.join(sector.split(' ')[:-1])
industry = ' '.join(industry.split(' ')[:-1])
desc = addr.children[0].children[0].children[-2].heading_text.replace('\n', ' ')
sector, industry, desc

('Information Technology',
 'Software',
 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide.')

In [146]:
ccs = addr.children[14].children
end = [c.heading_text for c in ccs].index('200-Day Moving Average')
ccs = ccs[1:end]
vals = [c.heading_text for c in ccs]

ccs = addr.children[2:15]
keys = [c.heading_text for c in ccs]

In [147]:
tbl = []

i = 0
for v in vals:
    k = None
    if '/' in v:
        if keys[i].startswith('Ex. Div'):
            i += 1
        if tbl[-1][0].startswith('Ex. Div'):
            tbl[-1][0] = f'{keys[i-2]} Date'
        i -= 1
        if not keys[i].endswith('Date'):
            k = f'{keys[i]} Date'    
    if k is None:
        k = keys[i]
    tbl.append([k, v])
    i += 1
tbl
key_stats = tbl

In [141]:
ccs = addr.children[27].children
esg_cats = [c.heading_text for c in addr.children[34].children[:4]]
ccs = addr.children[27].children[-8:-4]
esg_scores = [c.heading_text.split('\n')[0] for c in ccs]
esg = list(zip(esg_cats, esg_scores))
esg

[('Overall', 'LEADER'),
 ('Environment', 'LEADER'),
 ('Social', 'AVERAGE'),
 ('Governance', 'AVERAGE')]

In [168]:
ccs = D.elements[0].children[3].children[0].children[3].children
enames = ccs[2].heading_text.split('\n')[1:]
eacc = ccs[4].heading_text.split('\n')
escore = ccs[5].heading_text.split('\n')
equity_summary = list(zip(enames, eacc, escore))
equity_summary

[('Ford Equity Research (i)', '70', 'Neutral'),
 ('Refinitiv/Verus (i)', '87', 'Neutral'),
 ('Zacks Investment Research, Inc (i)', '82', 'Outperform'),
 ('ISS-EVA (i)', '94', 'Buy'),
 ('Jefferson Research (i)', '31', 'Neutral'),
 ('Trading Central (i)', '16', 'Buy'),
 ('McLean Capital Management (i)', '60', 'Neutral'),
 ('Argus Research A6/Quantitative  (i)', '32', 'Buy')]

In [177]:
ccs = D.elements[0].children[3].children[0].children[7].children

i = [i for i, c in enumerate(ccs) if c.heading_text.endswith('Summary Score Methodology (PDF).')][0] + 1

ns = [n.heading_text.split('\n') for n in ccs[i:]]
names = []
for n in ns:
    names.extend(n)
names

['Oppenheimer & Company',
 'RBC Capital Markets',
 'Raymond James',
 'KeyBanc Capital Markets',
 'Bernstein',
 'Stifel Nicolaus',
 'Jefferies',
 'CSFB',
 'Merrill Lynch',
 'UBS',
 'Morgan Stanley',
 'William Blair & Company',
 'Wedbush Morgan',
 'Citigroup Investment Research',
 'Deutsche Bank',
 'Piper Jaffray',
 'Argus Analyst',
 'Ford Equity Research (i)',
 'Refinitiv/Verus (i)',
 'Zacks Investment Research, Inc (i)',
 'ISS-EVA (i)',
 'Jefferson Research (i)',
 'Trading Central (i)',
 'McLean Capital Management (i)',
 'Argus Research A6/Quantitative  (i)',
 'I/B/E/S Estimates from Refinitiv']

In [179]:
ccs = D.elements[0].children[3].children[0].children[8].children
[c.heading_text for c in ccs]

['• Opinion used in Equity Summary Score',
 'Outperform',
 'Outperform',
 'Outperform',
 'Outperform',
 'Outperform',
 'Outperform',
 'Buy',
 'Buy',
 'Buy',
 'Buy',
 'Buy',
 'Buy',
 'Buy',
 'Buy',
 'Buy',
 'Outperform',
 'Outperform',
 'Buy',
 'Neutral',
 'Neutral',
 'Buy',
 'Neutral',
 'Buy',
 'Neutral',
 'Buy',
 '--',
 '--']

In [ ]:
ccs = D.elements[0].children[1].children
[c.heading_text for c in ccs]

In [30]:
# from pdf2image import convert_from_path
# pages = convert_from_path(path, 500)

In [31]:
# for i, page in enumerate(pages):
#     page.save(imdir/f'out{i}.jpg', 'JPEG')

In [32]:
page_no = 0

In [33]:
import pdfplumber

pdf_obj = pdfplumber.open(path)
page = pdf_obj.pages[page_no]
images_in_page = page.images
page_height = page.height
image_bbox = (image['x0'], page_height - image['y1'], image['x1'], page_height - image['y0'])
cropped_page = page.crop(image_bbox)
image_obj = cropped_page.to_image(resolution=400)
image_obj.save(path_to_save_image)

TypeError: 'JpegImageFile' object is not subscriptable

In [40]:
from pdfminer.layout import LAParams, LTTextBox
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator

rsrcmgr = PDFResourceManager()
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)


In [41]:
for page in PDFPage.get_pages(path.open('rb')):
#     print('Processing next page...')
    interpreter.process_page(page)
    layout = device.get_result()
    for lobj in layout:
        if isinstance(lobj, LTTextBox):
            x, y, text = lobj.bbox[0], lobj.bbox[3], lobj.get_text()
#             print(text)
            if '3-Year Price Performance' in text:
                print(text, x, y)
#             print('At %r is text: %s' % ((x, y), text))

3-Year Price Performance  Data provided by DataScope from Refinitiv
 174.96013996800002 596.7844774272


In [42]:
# import fitz # PyMuPDF
# import io
# from PIL import Image

In [43]:
# pdf_file = fitz.open(str(path))

In [44]:
# for page_index in range(len(pdf_file)):
#     # get the page itself
#     page = pdf_file[page_index]
#     image_list = page.getImageList()
#     # printing number of images found in this page
#     if image_list:
#         print(f"[+] Found a total of {len(image_list)} images in page {page_index}")
#     else:
#         print("[!] No images found on page", page_index)
#     for image_index, img in enumerate(page.getImageList(), start=1):
#         # get the XREF of the image
#         xref = img[0]
#         # extract the image bytes
#         base_image = pdf_file.extractImage(xref)
#         image_bytes = base_image["image"]
#         # get the image extension
#         image_ext = base_image["ext"]
#         # load it to PIL
#         image = Image.open(io.BytesIO(image_bytes))
#         # save it to local disk
#         image.save(open(f"image{page_index+1}_{image_index}.{image_ext}", "wb"))

In [45]:
from pdfstructure.hierarchy.parser import HierarchyParser
from pdfstructure.source import FileSource

parser = HierarchyParser() 

# specify source (that implements source.read())
source = FileSource(path.open('rb')) 

# analyse document and parse as nested data structure
document = parser.parse_pdf(source)

In [46]:
from pdfstructure.printer import PrettyStringPrinter

stringExporter = PrettyStringPrinter()
prettyString = stringExporter.print(document)
print(prettyString)



[Company Research Highlights ®]
	Report created on August 25, 2021

	[NVIDIA ORD SHS  NASDAQ:NVDA]

		[2788 San Tomas Expressway
		Santa Clara, CA 95051
		USA
		Telephone: 408 486 -2000
		www.nvidia.com]

			[Price on 8/24/2021:]

				[Market Capitalization:]
					$217.93
					$544.0 B
					Information Technology Sector
					Semiconductors & Semiconductor Equipment Industry
					Business Description  Data provided by S&P Compustat
					NVIDIA Corporation operates as a visual computing company worldwide. It operates in two
					segments, Graphics and Compute & Networking.
					3-Year Price Performance  Data provided by DataScope from Refinitiv
			Key Statistics
			Employee Count
			Institutional Ownership
			Total Revenue (TTM)

			[Revenue Growth]
				(TTM vs. Prior TTM)
			Enterprise Value
			Ex. Dividend Date
			Dividend
			P/E (TTM)
			Earning Yield (TTM)
			EPS (Adjusted TTM)
			Dividend Yield (Annualized)

			[Consensus EPS Estimate]
				(Q3 2021)

			[EPS Growth]
				(TTM vs. P

In [49]:
document.text

'Company Research Highlights ®\nReport created on August 25, 2021\nNVIDIA ORD SHS  NASDAQ:NVDA\n2788 San Tomas Expressway\nSanta Clara, CA 95051\nUSA\nTelephone: 408 486 -2000\nwww.nvidia.com\nPrice on 8/24/2021:\nMarket Capitalization:\n$217.93\n$544.0 B\nInformation Technology Sector\nSemiconductors & Semiconductor Equipment Industry\nBusiness Description  Data provided by S&P Compustat\nNVIDIA Corporation operates as a visual computing company worldwide. It operates in two\nsegments, Graphics and Compute & Networking.\n3-Year Price Performance  Data provided by DataScope from Refinitiv\nKey Statistics\nEmployee Count\nInstitutional Ownership\nTotal Revenue (TTM)\nRevenue Growth\n(TTM vs. Prior TTM)\nEnterprise Value\nEx. Dividend Date\nDividend\nP/E (TTM)\nEarning Yield (TTM)\nEPS (Adjusted TTM)\nDividend Yield (Annualized)\nConsensus EPS Estimate\n(Q3 2021)\nEPS Growth\n(TTM vs. Prior TTM)\n18,975\n66.5%\n$6,507.00\n7/31/2021\n+67.60%\n$537.1 B\n8/24/2021\n8/31/2021\n$0.040000\n0.0

In [7]:
import sys
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice, TagExtractor
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.cmapdb import CMapDB
from pdfminer.layout import LAParams
from pdfminer.image import ImageWriter
from pdfminer.high_level import extract_text

In [13]:
import io

In [8]:
rsrcmgr = PDFResourceManager(caching=True)

In [16]:
rsrcmgr = PDFResourceManager()
retstr = io.BytesIO()
codec = 'utf-8'
laparams = LAParams()
imagewriter = ImageWriter(str(imdir))

device = TextConverter(rsrcmgr, retstr, laparams=laparams, imagewriter=imagewriter)

In [17]:
with path.open('rb') as fp:
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0 #is for all
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

device.close()
sv = retstr.getvalue()
retstr.close()


In [12]:

# creating a pdf reader object 
pdfReader = PyPDF2.PdfFileReader(path.open('rb')) 
    
# printing number of pages in pdf file 
print(pdfReader.numPages) 
    
# creating a page object 
pageObj = pdfReader.getPage(1) 
    
# extracting text from page 
print(pageObj.extractText()) 
    
# closing the pdf file object 
# pdfFileObj.close()

4
Company Research Highlights®NVDANASDAQThis is not an investment recommendation from Fidelity Investments. The information
contained in this report is sourced from independent, third party providers.Price on 8/24/2021:$217.93Information Technology SectorMarket Capitalization:$544.0 BSemiconductors & Semiconductor Equipment Industry
NVIDIA ORD SHSThe content on this page is provided by third parties and not Fidelity. Fidelity did not prepare and does not endorse such content. All are third-party companies that are not affiliated with Fidelity.
See each section in this report for third-party content attribution and see page 4 for full disclosures.Page 2Report created on August 25, 2021Equity Summary Score is a weighted,aggregated view of opinions from the
independent research firms on Fidelity.com.
It uses the past accuracy of these firms in
determining the emphasis placed on any
individual opinion.
First Call Consensus Recommendation
is provided by I/B/E/S from Refinitiv, an
independen

In [13]:
pageObj.getContents()

{'/Filter': '/FlateDecode'}

In [14]:
pageObj.artBox

RectangleObject([0, 0, 612, 792])

In [17]:
from pdfminer.high_level import extract_text

In [20]:
text = extract_text(path)
print(text)

Company Research Highlights ®

Report created on August 25, 2021

NVIDIA ORD SHS  NASDAQ:NVDA
2788 San Tomas Expressway
Santa Clara, CA 95051
USA
Telephone: 408 486 -2000
www.nvidia.com

Price on 8/24/2021:
Market Capitalization:

$217.93
$544.0 B

Information Technology Sector
Semiconductors & Semiconductor Equipment Industry

Business Description  Data provided by S&P Compustat

NVIDIA Corporation operates as a visual computing company worldwide. It operates in two
segments, Graphics and Compute & Networking.

3-Year Price Performance  Data provided by DataScope from Refinitiv

Key Statistics

Employee Count

Institutional Ownership

Total Revenue (TTM)

Revenue Growth
(TTM vs. Prior TTM)

Enterprise Value

Ex. Dividend Date

Dividend

P/E (TTM)

Earning Yield (TTM)

EPS (Adjusted TTM)

Dividend Yield (Annualized)

Consensus EPS Estimate
(Q3 2021)

EPS Growth
(TTM vs. Prior TTM)

18,975

66.5%

$6,507.00
7/31/2021

+67.60%

$537.1 B
8/24/2021

8/31/2021

$0.040000

0.07%
8/24/2021

7

In [21]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer

In [24]:
for page_layout in extract_pages(path):
    break
page_layout

<LTPage(1) 0.000,0.000,612.000,792.000 rotate=0>

In [25]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import HTMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import BytesIO


def convert_pdf_to_html(path):
    rsrcmgr = PDFResourceManager()
    retstr = BytesIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = HTMLConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0 #is for all
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    fp.close()
    device.close()
    str = retstr.getvalue()
    retstr.close()
    return str

test = convert_pdf_to_html(path)

In [27]:
with open('test.html', 'wb') as f:
    f.write(test)

In [ ]:
for page in doc.iter_pages():
    im = page.images[0].as_pil()  # requires pillow
    display(im)

In [15]:
document.style_distribution

In [17]:
document.metadata

defaultdict(str,
            {'style_distribution': <pdfstructure.analysis.styledistribution.StyleDistribution at 0x296350c74c0>,
             'filename': 'msft_highlights.pdf'})

In [ ]:
document.

In [26]:
for page in doc.iter_pages():
    im = page.images[0].as_pil()  # requires pillow
    display(im)

NameError: name 'doc' is not defined

In [20]:
for shape in page.shapes.iter_in_bbox((0, 0, 100, 200)):
    print(shape.path, shape.fill.color.as_rgb())
im = page.images[0].as_pil()  # requires pillow
im.show()

NameError: name 'page' is not defined

In [6]:
print(prettyString)



[Company Research Highlights ®]
	Report created on August 25, 2021

	[NVIDIA ORD SHS  NASDAQ:NVDA]

		[2788 San Tomas Expressway
		Santa Clara, CA 95051
		USA
		Telephone: 408 486 -2000
		www.nvidia.com]

			[Price on 8/24/2021:]

				[Market Capitalization:]
					$217.93
					$544.0 B
					Information Technology Sector
					Semiconductors & Semiconductor Equipment Industry
					Business Description  Data provided by S&P Compustat
					NVIDIA Corporation operates as a visual computing company worldwide. It operates in two
					segments, Graphics and Compute & Networking.
					3-Year Price Performance  Data provided by DataScope from Refinitiv
			Key Statistics
			Employee Count
			Institutional Ownership
			Total Revenue (TTM)

			[Revenue Growth]
				(TTM vs. Prior TTM)
			Enterprise Value
			Ex. Dividend Date
			Dividend
			P/E (TTM)
			Earning Yield (TTM)
			EPS (Adjusted TTM)
			Dividend Yield (Annualized)

			[Consensus EPS Estimate]
				(Q3 2021)

			[EPS Growth]
				(TTM vs. P

In [13]:
document.elements[0].children[0].heading_text

'Report created on August 25, 2021'